# Data cleaning

## 1.Importing libraries and data.

In [2]:
import pandas as pd
import re
import numpy as np

In [3]:
fotocasa = pd.read_csv('csv/fotocasa.csv')

In [4]:
fotocasa.head()

,Unnamed: 0,description,address,price
0,0,3 habs.1 baño60 m²con ascensor,PisoHorta - Guinardóahora,750 € /mes
1,1,2 habs.1 baño59 m²con ascensor,"PisoCarrer del Cardenal Tedeschini, Sant Andre...",770 € /mes
2,2,1 hab.1 baño30 m²con ascensor y terraza,"PisoCarrer del Pare Laínez, Gràciaahora",1.300 € /mes
3,3,1 hab.1 baño70 m²con ascensor y terraza,"PisoPasseig D'isabel II, 2, Barcelona, España,...",2.800 € /mes
4,4,2 habs.1 baño44 m²con ascensor,"PisoCarrer de Cienfuegos, Sant Andreuahora",720 € /mes


In [5]:
#Creating a new column to separate all the description column information.
fotocasa['rooms'] = fotocasa['description']
fotocasa['bathroom'] = fotocasa['description']
fotocasa['lift'] = fotocasa['description']
fotocasa['terrace'] = fotocasa['description']
fotocasa['square_meters'] = fotocasa['description']

## 2. Cleaning columns.

### 2.1 Clean rooms.

In [6]:
def rooms(row):
    return re.findall('.*\.',row)

In [7]:
def fill_space(row):
    if row == []:
        return ['0']
    else:
        return row

In [8]:
def list_clean(row):
    return row[0]

In [9]:
fotocasa['rooms'] = fotocasa['rooms'].apply(rooms)
fotocasa['rooms'] = fotocasa['rooms'].apply(fill_space)
fotocasa['rooms'] = fotocasa['rooms'].apply(list_clean)

### 2.2 Clean bathroom.

In [10]:
def bathroom(row):
    return re.findall('[\d\s]*baño',row)

In [11]:
def fill_space(row):
    if row == []:
        return ['1']
    else:
        return row

In [12]:
fotocasa['bathroom'] = fotocasa['bathroom'].apply(bathroom)
fotocasa['bathroom'] = fotocasa['bathroom'].apply(fill_space)
fotocasa['bathroom'] = fotocasa['bathroom'].apply(list_clean)

### 2.3 Clean lift.

In [13]:
def lift(row):
    return re.findall('\w*cens\w*',row)

In [14]:
def fill_space(row):
    if row == []:
        return False
    else:
        return True

In [15]:
fotocasa['lift'] = fotocasa['lift'].apply(lift)
fotocasa['lift'] = fotocasa['lift'].apply(fill_space)

### 2.4 Clean terrace.

In [16]:
def terrace(row):
    return re.findall('\w*rraz\w*',row)

In [17]:
def fill_space(row):
    if row == []:
        return False
    else:
        return True

In [18]:
fotocasa['terrace'] = fotocasa['terrace'].apply(terrace)
fotocasa['terrace'] = fotocasa['terrace'].apply(fill_space)

### 2.5 Clean square_meters.

In [19]:
def square_meters(row):
    return re.findall('[\d\s]*m²',row)

In [20]:
def fill_space(row):
    if row == []:
        return ['0']
    else:
        return row

In [21]:
fotocasa['square_meters'] = fotocasa['square_meters'].apply(square_meters)
fotocasa['square_meters'] = fotocasa['square_meters'].apply(fill_space)
fotocasa['square_meters'] = fotocasa['square_meters'].apply(list_clean)

### 2.6 Clean prices.

In [22]:
def price(row):
    return re.findall('.*mes',row)

In [23]:
def price_coma(row):
    return re.sub(',','',row)

In [24]:
def fill_space(row):
    if row == []:
        return ['0']
    else:
        return row

In [25]:
fotocasa['price'] = fotocasa['price'].apply(price)
fotocasa['price'] = fotocasa['price'].apply(fill_space)
fotocasa['price'] = fotocasa['price'].apply(list_clean)
fotocasa['price'] = list(map(lambda x: x.replace('.',''),fotocasa['price']))

## 3. More cleaning of the data set.

### 3.1 To get just numbers of numerical columns and make them numerical.

In [26]:
num_cols = ['price','rooms','bathroom','square_meters']

In [27]:
def get_numbers(row):
    return re.findall('[\d.]{1,7}',row)

In [28]:
for column in num_cols:
    fotocasa[column] = fotocasa[column].apply(get_numbers)

In [29]:
for column in num_cols:
    fotocasa[column] = fotocasa[column].apply(list_clean)

In [30]:
for column in num_cols:
    fotocasa[column] = pd.to_numeric(fotocasa[column])

### 3.2 Remiving rows without price or square meteres and some outliers.

In [31]:
fotocasa = fotocasa.drop(['Unnamed: 0','description'],axis=1)
fotocasa = fotocasa[fotocasa['price']!=0]
fotocasa = fotocasa[fotocasa['square_meters']!=0]

In [32]:
fotocasa = fotocasa[fotocasa['price']<20000]
fotocasa = fotocasa[fotocasa['square_meters']<700]

### 3.3 Creating real state.

In [33]:
fotocasa['real_state'] = fotocasa['address']

In [34]:
def flat_house(row):
    return re.findall('([\w]*[ABCDEFGHIJKLMNÑOPQRSTUVWXYZ])\w',row)

In [35]:
def flat_house1(row):
    return row[0]

In [36]:
def flat_house2(row):
    return row[:-1]

In [37]:
fotocasa['real_state'] = fotocasa['real_state'].apply(flat_house)
fotocasa['real_state'] = fotocasa['real_state'].apply(flat_house1)
fotocasa['real_state'] = fotocasa['real_state'].apply(flat_house2)

In [38]:
fotocasa = fotocasa[fotocasa['real_state']!='']

In [39]:
fotocasa.reset_index(inplace=True,drop=True)

### 3.4 Creating address.

In [40]:
fotocasa['address1'] = fotocasa['address']

In [41]:
def clean_ahora(row):
    return re.sub('ahora.*','',row)

In [42]:
def clean_hace(row):
    return re.sub('hace.*','',row)

In [43]:
fotocasa['address1'] = fotocasa['address1'].apply(clean_ahora)
fotocasa['address1'] = fotocasa['address1'].apply(clean_hace)

In [44]:
for row in range(len(fotocasa['address1'])):
    fotocasa.loc[row,'address1'] = fotocasa.loc[row,'address1'][len(fotocasa.loc[row,'real_state']):]

In [45]:
fotocasa = fotocasa.drop(['address'],axis=1)

In [46]:
fotocasa['neighborhood'] = fotocasa['address1']

In [47]:
fotocasa.head()

,price,rooms,bathroom,lift,terrace,square_meters,real_state,address1,neighborhood
0,750,3,1,True,False,60,Piso,Horta - Guinardó,Horta - Guinardó
1,770,2,1,True,False,59,Piso,"Carrer del Cardenal Tedeschini, Sant Andreu","Carrer del Cardenal Tedeschini, Sant Andreu"
2,1300,1,1,True,True,30,Piso,"Carrer del Pare Laínez, Gràcia","Carrer del Pare Laínez, Gràcia"
3,2800,1,1,True,True,70,Piso,"Passeig D'isabel II, 2, Barcelona, España, Ciu...","Passeig D'isabel II, 2, Barcelona, España, Ciu..."
4,720,2,1,True,False,44,Piso,"Carrer de Cienfuegos, Sant Andreu","Carrer de Cienfuegos, Sant Andreu"


In [48]:
fotocasa['neighborhood'] = fotocasa['neighborhood'].str.split(',').str[-1]
fotocasa['neighborhood'] = fotocasa['neighborhood'].str.split('-').str[-1]

In [49]:
fotocasa['address1'] = fotocasa['address1'].str.split(',').str[0]
fotocasa['address1'] = fotocasa['address1'].str.split('-').str[0]

In [50]:
fotocasa['neighborhood'] = list(map(lambda x: x.strip(),fotocasa['neighborhood']))

In [51]:
fotocasa['real_state'] = fotocasa['real_state'].map({'Piso':'flat','Ático':'attic','Apartamento':'apartment','Estudio':'study'})

### 3.5 Square meters.

In [52]:
neighborhood = list(fotocasa['neighborhood'].unique())

In [53]:
fotocasa['square_meters_price'] = fotocasa['price']/fotocasa['square_meters']

### 3.6 Clenaning white spaces.

In [54]:
def clean_1whitespace_left(row):
    if row[0]== ' ':
        return row[1:]
    else: 
        return row

In [55]:
def clean_2whitespace_left(row):
    if row[:2]== '  ':
        return row[2:]
    else: 
        return row

In [56]:
def clean_3whitespace_left(row):
    if row[:3]== '   ':
        return row[3:]
    else: 
        return row

In [57]:
def clean_1whitespace_right(row):
    if row[-1]== ' ':
        return row[:-1]
    else: 
        return row

In [58]:
def clean_2whitespace_right(row):
    if row[-2:]== '  ':
        return row[:-2]
    else: 
        return row

In [59]:
def clean_3whitespace_right(row):
    if row[-3:]== '   ':
        return row[:-3]
    else: 
        return row

In [60]:
fotocasa['address1'] = fotocasa['address1'].apply(clean_1whitespace_left)
fotocasa['address1'] = fotocasa['address1'].apply(clean_2whitespace_left)
fotocasa['address1'] = fotocasa['address1'].apply(clean_3whitespace_left)
fotocasa['address1'] = fotocasa['address1'].apply(clean_1whitespace_right)
fotocasa['address1'] = fotocasa['address1'].apply(clean_2whitespace_right)
fotocasa['address1'] = fotocasa['address1'].apply(clean_3whitespace_right)

### 3.7 More cleaning of address and neighborhood.

In [61]:
neighborhood1 = list(fotocasa['neighborhood'].unique())

In [62]:
def cleaning_adress_neighborhood1(row):
    if row in neighborhood1:
        return 'Another'
    else:
        return row

In [63]:
fotocasa['address1'] = fotocasa['address1'].apply(cleaning_adress_neighborhood1)

In [64]:
fotocasa['neighborhood'] = fotocasa['neighborhood'].map({'Eixample':'Eixample','Guinardó':'Horta- Guinardo', 
                              'Sant Gervasi': 'Sarria-Sant Gervasi','Ciutat Vella':'Ciutat Vella','Gràcia':'Gràcia',
                              'Sant Martí':'Sant Martí','Montjuïc':'Sants-Montjuïc','Marques de Mul':'Les Corts',
                             'Les Corts':'Les Corts','Sant Andreu':'Sant Andreu','Nou Barris':'Nou Barris'})

In [65]:
neighborhood2 = ['Horta','Sarria','Sarrià','Ciutat','Sant','Sants','Les','Nou']

In [66]:
def cleaning_adress_neighborhood2(row):
    if row in neighborhood2:
        return 'Another'
    else:
        return row

In [67]:
fotocasa['address1'] = fotocasa['address1'].apply(cleaning_adress_neighborhood2)

### 3.8 Drop address.

In [68]:
#For the final prototipe it does not worth cleaning it.
fotocasa = fotocasa.drop(['address1'],axis=1)

In [69]:
fotocasa.to_csv('csv/fotocasa_clean.csv')

## 4. Pivot tables.

### 4.1 Lift and terrace price.

In [70]:
terrace = fotocasa[fotocasa['terrace']==True]

In [71]:
lift = fotocasa[fotocasa['lift']==True]

In [72]:
A = terrace.pivot_table(index = ['neighborhood'], values = ['price'], aggfunc = ['mean']).reset_index()

In [73]:
B = lift.pivot_table(index = ['neighborhood'], values = ['price'], aggfunc = ['mean']).reset_index()

In [74]:
C = fotocasa.pivot_table(index = ['neighborhood'], values = ['price'], aggfunc = ['mean']).reset_index()

#### 4.1.1 Terrace price.

In [75]:
AC = pd.merge(left = A,
                         right = C,
                         how = 'inner', 
                         left_on = 'neighborhood', 
                         right_on= 'neighborhood')

/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [76]:
AC['terrace_price'] = AC['mean_x']-AC['mean_y']

In [77]:
AC

,neighborhood,mean_x,mean_y,terrace_price
,,price,price,
0,Ciutat Vella,1581.642458,1205.523810,376.118649
1,Eixample,1832.644211,1592.147022,240.497188
2,Gràcia,1261.079137,1185.434593,75.644544
3,Horta- Guinardo,1051.820896,903.453968,148.366927
4,Les Corts,2093.320513,1610.748092,482.572421
5,Nou Barris,834.950000,807.921053,27.028947
6,Sant Andreu,971.916667,899.657303,72.259363
7,Sant Martí,1928.795918,1434.876823,493.919095
8,Sants-Montjuïc,1034.760000,1028.233898,6.526102


In [78]:
AC['terrace_price'].mean()

255.93821769695415

#### 4.1.2 Lift price.

In [80]:
BC = pd.merge(left = B,
                         right = C,
                         how = 'inner', 
                         left_on = 'neighborhood', 
                         right_on= 'neighborhood')

/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [81]:
BC['lift_price'] = BC['mean_x']-BC['mean_y']
BC

,neighborhood,mean_x,mean_y,lift_price
,,price,price,
0,Ciutat Vella,1343.878089,1205.523810,138.354279
1,Eixample,1477.358783,1592.147022,-114.788239
2,Gràcia,1209.838942,1185.434593,24.404349
3,Horta- Guinardo,941.154696,903.453968,37.700728
4,Les Corts,1669.705618,1610.748092,58.957526
5,Nou Barris,842.442857,807.921053,34.521805
6,Sant Andreu,900.389313,899.657303,0.732010
7,Sant Martí,1433.653017,1434.876823,-1.223806
8,Sants-Montjuïc,981.391176,1028.233898,-46.842722


In [255]:
BC['lift_price'].mean()

20.00661913744394

### 4.2 Price comparison fotocasa vs lloguer.

In [256]:
fotocasa_price = fotocasa.pivot_table(index = ['neighborhood'], 
                                      values = ['square_meters_price'], aggfunc = ['mean']).reset_index()

In [311]:
lloguer_price = pd.read_csv('csv/lloguer.csv')
lloguer_price = lloguer_price.drop(['FID','ID','DISTRICTE','CONTRAC','CONT1000H','VARLLOG','LLOGTRIM','GEOMETRY'],axis=1)
lloguer_price['DISTR_BCN']= lloguer_price['DISTR_BCN'].map({'Eixample':'Eixample','Horta-Guinardó':'Horta- Guinardo', 
                              'Sarrià-Sant Gervasi': 'Sarria-Sant Gervasi','Ciutat Vella':'Ciutat Vella','Gràcia':'Gràcia',
                              'Sant Martí':'Sant Martí','Sants-Montjuïc':'Sants-Montjuïc',
                             'Les Corts':'Les Corts','Sant Andreu':'Sant Andreu','Nou Barris':'Nou Barris'})
lloguer_price['PREUM2'] = lloguer_price['LLOGUER']/lloguer_price['SUPMITJ']

In [312]:
price_comparison = pd.merge(left = fotocasa_price,
                         right = lloguer_price,
                         how = 'inner', 
                         left_on = 'neighborhood', 
                         right_on= 'DISTR_BCN')

/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [313]:
price_comparison = price_comparison.drop(['DISTR_BCN','TRAMLLOG','LLOGUER','SUPMITJ','PREUM2TRIM','VARCONTR'],axis=1)

In [316]:
price_comparison = price_comparison.rename(columns={('mean', 'square_meters_price'):'fotocasa_price',
                                 ('neighborhood',''):'neighborhood','PREUM2':'lloguer_price'})

In [322]:
price_comparison['diference']= round(100-((price_comparison['lloguer_price']*100)/price_comparison['fotocasa_price']),0)

In [329]:
price_comparison['neighborhood']=price_comparison['neighborhood'].map({'Eixample':'Eixample','Horta- Guinardo':'Horta-Guinardó', 
                              'Sarria-Sant Gervasi':'Sarrià-Sant Gervasi','Ciutat Vella':'Ciutat Vella','Gràcia':'Gràcia',
                              'Sant Martí':'Sant Martí','Sants-Montjuïc':'Sants-Montjuïc',
                             'Les Corts':'Les Corts','Sant Andreu':'Sant Andreu','Nou Barris':'Nou Barris'})

In [331]:
price_comparison.to_csv('csv/price_comparison.csv')

In [411]:
fotocasa.pivot_table(index = ['real_state'], values = ['square_meters_price'], aggfunc = ['mean'])

,mean
,square_meters_price
real_state,
apartment,28.969599
attic,19.902672
flat,15.755932
study,19.514038


In [1938]:
fotocasa.pivot_table(index = ['neighborhood','real_state'], values = ['square_meters_price'], aggfunc = ['mean'])

mean
                               square_meters_price
neighborhood        real_state                    
Ciutat Vella        apartment            25.818935
                    attic                23.136896
                    flat                 16.933190
                    study                19.802116
Eixample            apartment            33.225204
                    attic                20.405715
                    flat                 15.981306
                    study                21.561559
Gràcia              apartment            30.462194
                    attic                20.365350
                    flat                 15.424873
                    study                18.452523
Horta- Guinardo     apartment            18.615852
                    attic                13.166584
                    flat                 13.128600
                    study                17.716288
Les Corts           apartment            21.426308
                    attic                18.799646
                    flat                 15.139123
                    study                13.117716
Nou Barris          apartment            25.384615
                    attic                13.754236
                    flat                 12.243987
Sant Andreu         apartment            25.951816
                    attic                13.499580
                    flat                 12.607824
                    study                16.325000
Sant Martí          apartment            33.001068
                    attic                27.776892
                    flat                 15.825835
                    study                22.220787
Sants-Montjuïc      apartment            32.065966
                    attic                18.514327
                    flat                 14.795736
                    study                17.531891
Sarria-Sant Gervasi apartment            23.044091
                    attic                19.737263
                    flat                 16.534916
                    study                20.954568